In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.tools import BaseTool
from langchain_core.runnables import Runnable, RunnableConfig
from langchain_core.language_models import LanguageModelLike
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint import MemorySaver
import operator
# Define the state
class MetaPromptingState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]
    expert_instructions: str
    expert_response: str
    is_last_step: bool


In [20]:

from utils.prompt_loader import load_markdown_prompt
from tools.demo_tools import WebSearch
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

# Load meta-prompter instructions
META_PROMPTER_INSTRUCTIONS = load_markdown_prompt("../prompts/meta-prompter.md")

# Create the language model
model = ChatOpenAI(model_name="gpt-4o", temperature=0)

tools = [TavilySearchResults]


In [21]:
# Create the meta-prompter node
def meta_prompter(state: MetaPromptingState, config: RunnableConfig):
    messages = state['messages']
    response = model.invoke(
        [
            HumanMessage(content=META_PROMPTER_INSTRUCTIONS),
            *messages
        ],
        config
    )
    return {
        "expert_instructions": response.content,
        "messages": [AIMessage(content=f"Meta-Prompter: {response.content}")]
    }

# Create the expert node
def expert(state: MetaPromptingState, config: RunnableConfig):
    expert_instructions = state['expert_instructions']
    response = model.invoke(
        [HumanMessage(content=expert_instructions)],
        config
    )
    return {
        "expert_response": response.content,
        "messages": [AIMessage(content=f"Expert: {response.content}")]
    }


In [22]:
# Define the function to determine whether to continue or end
def should_continue(state: MetaPromptingState) -> Literal["continue", "use_tool", "end"]:
    last_message = state['messages'][-1]
    if ">> FINAL ANSWER:" in last_message.content:
        return "end"
    elif any(tool.name in last_message.content for tool in tools):
        return "use_tool"
    else:
        return "continue"


In [23]:
# Create the tool node
tool_node = ToolNode(tools)

# Create the graph
workflow = StateGraph(MetaPromptingState)

# Add nodes
workflow.add_node("meta_prompter", meta_prompter)
workflow.add_node("expert", expert)
workflow.add_node("tools", tool_node)

# Set entry point
workflow.set_entry_point("meta_prompter")

# Add edges
workflow.add_conditional_edges(
    "meta_prompter",
    should_continue,
    {
        "continue": "expert",
        "use_tool": "tools",
        "end": END
    }
)
workflow.add_edge("expert", "meta_prompter")
workflow.add_edge("tools", "expert")

# Compile the graph
agent = workflow.compile()


TypeError: Object of type 'ModelMetaclass' is not JSON serializable

In [ ]:


result = agent.invoke({"messages": [HumanMessage(content="Write a paragraph describing the current weather in Conway Arkansas as if you were a 3 year old")]})
print(result)